### Importação de configurações e funções

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *
from Utils.bronze_csv_loader import *
from Utils.utils import *

apply_storage_config(spark)

In [0]:
from Utils.incremental_bronze import *
from pyspark.sql.functions import (
    input_file_name,
    current_timestamp,
    lit,
    col,
    count
)
from pyspark.sql.types import StructType, StructField, StringType
import re, os

In [0]:
%sql
SELECT * FROM bronze.arquivos_processados_balancacomercial

### Criação da tabela de controle de arquivos

In [0]:
def criar_tabela_controle_arquivos():
    """
hive_metastore.bronze.arquivos_processados_cnpj    Cria tabela para rastrear arquivos CSV já processados
    """
    spark.sql("""
        CREATE TABLE IF NOT EXISTS bronze.arquivos_processados_balancacomercial (
            caminho_arquivo STRING,
            nome_arquivo STRING,
            tipo_operacao STRING,
            ano STRING,
            mes STRING,
            data_processamento TIMESTAMP,
            num_registros BIGINT
        )
        USING DELTA
    """)
    print("Tabela de controle de arquivos criada!")

criar_tabela_controle_arquivos()

### Funções auxiliares

In [0]:
def listar_csvs_recursivo(path_base):
    """
    Lista todos os arquivos CSV recursivamente a partir de um caminho base
    """
    arquivos = []

    def _ls(path):
        for item in dbutils.fs.ls(path):
            if item.isDir():
                _ls(item.path)
            else:
                if item.name.lower().endswith(".csv"):
                    arquivos.append({
                        "path": item.path,
                        "name": item.name
                    })

    _ls(path_base)
    return arquivos

In [0]:
def obter_arquivos_novos(input_path):
    """
    Filtra apenas arquivos que ainda não foram processados
    Usa o nome do arquivo como chave de controle
    """
    arquivos = listar_csvs_recursivo(input_path)

    try:
        processados = spark.sql("""
            SELECT DISTINCT nome_arquivo
            FROM bronze.arquivos_processados_balancacomercial
        """).collect()
        processados_set = {r.nome_arquivo for r in processados}
    except:
        processados_set = set()

    novos = [a for a in arquivos if a["name"] not in processados_set]

    print(f"Total encontrados: {len(arquivos)}")
    print(f"Novos para processar: {len(novos)}")

    return novos

In [0]:
def registrar_arquivo_processado(arquivo, df, tipo_operacao):
    """
    Registra arquivo processado na tabela de controle
    Executa APENAS após ingestão bem-sucedida
    """
    num_registros = df.count()

    ano = df.select("ano").first()[0] if "ano" in df.columns else "N/A"
    mes = df.select("mes").first()[0] if "mes" in df.columns else "N/A"

    spark.sql(f"""
        INSERT INTO bronze.arquivos_processados_balancacomercial
        VALUES (
            '{arquivo["path"]}',
            '{arquivo["name"]}',
            '{tipo_operacao}',
            '{ano}',
            '{mes}',
            current_timestamp(),
            {num_registros}
        )
    """)

### Atribuição de variáveis e paths

In [0]:
TIPO_OPERACAO_EXP = "EXP"
TIPO_OPERACAO_IMP = "IMP"
NCM = "NCM"
TB_AUX = "TB_AUX"

input_path = f"{balanca_comercial_path}"

output_path_exp = f"{bronze_path}balancacomercial/{TIPO_OPERACAO_EXP.lower()}"
output_path_imp = f"{bronze_path}balancacomercial/{TIPO_OPERACAO_IMP.lower()}"
output_path_ncm = f"{bronze_path}balancacomercial/ncm"
output_path_tb_aux = f"{bronze_path}balancacomercial/tb_aux"

print(f"Input path: {input_path}")
print(f"Output EXP: {output_path_exp}")
print(f"Output IMP: {output_path_imp}")
print(f"Output NCM: {output_path_ncm}")

### Ingestão EXP (Exportação)

In [0]:
expected_cols_exp = [
    "CO_ANO","CO_MES","CO_NCM","CO_UNID","CO_PAIS",
    "SG_UF_NCM","CO_VIA","CO_URF","QT_ESTAT","KG_LIQUIDO","VL_FOB"
]

schema_exp = StructType([StructField(c, StringType(), True) for c in expected_cols_exp])

# Leitura com read_csv_with_quotes + path_glob_filter
df_raw_exp, df_exp_corrupt, exp_cols = read_csv_with_quotes(
    spark=spark,
    input_path=input_path,
    delimiter=";",
    encoding="iso-8859-1",
    recursive=True,
    path_glob_filter="EXP_[0-9][0-9][0-9][0-9].csv",
    header=True,
    schema=schema_exp,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/exp",
    quarantine_mode="append"
)

print(f"EXP lidos: {df_raw_exp.count()} registros")
print(f"EXP corrompidos: {df_exp_corrupt.count() if df_exp_corrupt is not None else 0} registros")

In [0]:
from pyspark.sql.functions import regexp_extract, col, input_file_name# Aplicar incremental APÓS leitura

df_processados_exp = spark.table("bronze.arquivos_processados_balancacomercial") \
    .filter(col("tipo_operacao") == TIPO_OPERACAO_EXP) \
    .select("nome_arquivo") \
    .distinct()

df_novos_exp = (
    df_raw_exp
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

# Filtro incremental por nome
df_novos_exp = df_novos_exp.join(
    df_processados_exp,
    df_novos_exp.file_name_only == df_processados_exp.nome_arquivo,
    "left_anti"
)

print(f"EXP novos para processar: {df_novos_exp.count()}")

In [0]:
df_bronze_exp = (
    df_novos_exp
    .withColumn("ano", col("CO_ANO").cast("int"))
    .withColumn("mes", col("CO_MES"))
    .withColumn("tipo_operacao", lit(TIPO_OPERACAO_EXP))
    .withColumn("ingestion_dt", current_timestamp())
)

In [0]:
# Enriquecimento Bronze
df_bronze_exp = (
    df_novos_exp
    .withColumn("ano", col("CO_ANO").cast("int"))
    .withColumn("mes", col("CO_MES"))
    .withColumn("tipo_operacao", lit(TIPO_OPERACAO_EXP))
    .withColumn("ingestion_dt", current_timestamp())
)

# Escrita incremental (append)
(
    df_bronze_exp
    .write
    .format("delta")
    .mode("append")
    .partitionBy("ano", "mes")
    .save(output_path_exp)
)

print(f"EXP escritos em Bronze: {df_bronze_exp.count()} registros")

In [0]:
# Registrar arquivos processados (apenas os novos)
df_registro_exp = (
    df_bronze_exp
    .groupBy("origin_file_name", "file_name_only", "ano", "mes")
    .agg(count("*").alias("num_registros"))
)

for row in df_registro_exp.collect():
    spark.sql(f"""
        INSERT INTO bronze.arquivos_processados_balancacomercial
        VALUES (
            '{row.origin_file_name}',
            '{row.file_name_only}',
            '{TIPO_OPERACAO_EXP}',
            '{row.ano}',
            '{row.mes}',
            current_timestamp(),
            {row.num_registros}
        )
    """)

print(" EXP registrados em controle")

### Ingestão IMP (Importação)

In [0]:
expected_cols_imp = [
    "CO_ANO","CO_MES","CO_NCM","CO_UNID","CO_PAIS",
    "SG_UF_NCM","CO_VIA","CO_URF","QT_ESTAT","KG_LIQUIDO","VL_FOB","VL_FRETE","VL_SEGURO"
]

schema_imp = StructType([StructField(c, StringType(), True) for c in expected_cols_imp])

# Leitura com read_csv_with_quotes + path_glob_filter
df_raw_imp, df_imp_corrupt, imp_cols = read_csv_with_quotes(
    spark=spark,
    input_path=input_path,
    delimiter=";",
    encoding="iso-8859-1",
    recursive=True,
    path_glob_filter="IMP_[0-9][0-9][0-9][0-9].csv",
    header=True,
    schema=schema_imp,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/imp",
    quarantine_mode="append"
)

print(f"IMP lidos: {df_raw_imp.count()} registros")
print(f"IMP corrompidos: {df_imp_corrupt.count() if df_imp_corrupt is not None else 0} registros")

In [0]:
# Aplicar incremental APÓS leitura
df_processados_imp = spark.table("bronze.arquivos_processados_balancacomercial") \
    .filter(col("tipo_operacao") == TIPO_OPERACAO_IMP) \
    .select("nome_arquivo") \
    .distinct()

df_novos_imp = (
    df_raw_imp
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

# Filtro incremental por nome
df_novos_imp = df_novos_imp.join(
    df_processados_imp,
    df_novos_imp.file_name_only == df_processados_imp.nome_arquivo,
    "left_anti"
)

print(f"IMP novos para processar: {df_novos_imp.count()}")

In [0]:
# Enriquecimento Bronze
df_bronze_imp = (
    df_novos_imp
    .withColumn("ano", col("CO_ANO").cast("int"))
    .withColumn("mes", col("CO_MES"))
    .withColumn("tipo_operacao", lit(TIPO_OPERACAO_IMP))
    .withColumn("ingestion_dt", current_timestamp())
)

# Escrita incremental (append)
(
    df_bronze_imp
    .write
    .format("delta")
    .mode("append")
    .partitionBy("ano", "mes")
    .save(output_path_imp)
)

print(f"IMP escritos em Bronze: {df_bronze_imp.count()} registros")

In [0]:
# Registrar arquivos processados (apenas os novos)
df_registro_imp = (
    df_bronze_imp
    .groupBy("origin_file_name", "file_name_only", "ano", "mes")
    .agg(count("*").alias("num_registros"))
)

for row in df_registro_imp.collect():
    spark.sql(f"""
        INSERT INTO bronze.arquivos_processados_balancacomercial
        VALUES (
            '{row.origin_file_name}',
            '{row.file_name_only}',
            '{TIPO_OPERACAO_IMP}',
            '{row.ano}',
            '{row.mes}',
            current_timestamp(),
            {row.num_registros}
        )
    """)

print("IMP registrados em controle")

### Ingestão NCM

In [0]:
expected_cols_ncm = [
    "CO_NCM","CO_UNID","CO_SH6","CO_PPE","CO_PPI","CO_FAT_AGREG",
    "CO_CUCI_ITEM","CO_CGCE_N3","CO_SIIT","CO_ISIC_CLASSE","CO_EXP_SUBSET",
    "NO_NCM_POR","NO_NCM_ESP","NO_NCM_ING"
]

schema_ncm = StructType([StructField(c, StringType(), True) for c in expected_cols_ncm])

# Leitura com read_csv_with_quotes + path_glob_filter
df_raw_ncm, df_ncm_corrupt, ncm_cols = read_csv_with_quotes(
    spark=spark,
    input_path=input_path,
    delimiter=";",
    encoding="iso-8859-1",
    recursive=True,
    path_glob_filter="NCM.csv",
    header=True,
    schema=schema_ncm,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/ncm",
    quarantine_mode="append"
)

print(f"NCM lidos: {df_raw_ncm.count()} registros")
print(f"NCM corrompidos: {df_ncm_corrupt.count() if df_ncm_corrupt is not None else 0} registros")

In [0]:
# Aplicar incremental APÓS leitura
df_processados_ncm = spark.table("bronze.arquivos_processados_balancacomercial") \
    .filter(col("tipo_operacao") == NCM) \
    .select("nome_arquivo") \
    .distinct()

df_novos_ncm = (
    df_raw_ncm
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

# Filtro incremental por nome
df_novos_ncm = df_novos_ncm.join(
    df_processados_ncm,
    df_novos_ncm.file_name_only == df_processados_ncm.nome_arquivo,
    "left_anti"
)

print(f"NCM novos para processar: {df_novos_ncm.count()}")

In [0]:
# Enriquecimento Bronze
df_bronze_ncm = (
    df_novos_ncm
    .withColumn("tipo_operacao", lit(NCM))
    .withColumn("ingestion_dt", current_timestamp())
)

# Escrita incremental (append)
(
    df_bronze_ncm
    .write
    .format("delta")
    .mode("append")
    .partitionBy("ingestion_dt")
    .save(output_path_ncm)
)

print(f"NCM escritos em Bronze: {df_bronze_ncm.count()} registros")

In [0]:
# Registrar arquivos processados (apenas os novos)
df_registro_ncm = (
    df_bronze_ncm
    .groupBy("origin_file_name", "file_name_only")
    .agg(count("*").alias("num_registros"))
)

for row in df_registro_ncm.collect():
    spark.sql(f"""
        INSERT INTO bronze.arquivos_processados_balancacomercial
        VALUES (
            '{row.origin_file_name}',
            '{row.file_name_only}',
            '{NCM}',
            'N/A',
            'N/A',
            current_timestamp(),
            {row.num_registros}
        )
    """)

print("NCM registrados em controle")

### Ingestão Tabelas Auxiliares

In [0]:
# Listar arquivos que não são EXP, IMP ou NCM
arquivos_csv = []

for f in dbutils.fs.ls(input_path):
    nome = f.name

    if not nome.lower().endswith(".csv"):
        continue

    if re.search(r"IMP_[0-9]{4}\.csv$", nome):
        continue

    if re.search(r"EXP_[0-9]{4}\.csv$", nome):
        continue

    if re.search(r"(?i)ncm\.csv$", nome):
        continue

    arquivos_csv.append(f.path)

print(f"Arquivos auxiliares encontrados: {len(arquivos_csv)}")

In [0]:
# Filtrar apenas arquivos novos
df_processados_aux = spark.table("bronze.arquivos_processados_balancacomercial") \
    .filter(col("tipo_operacao") == TB_AUX) \
    .select("nome_arquivo") \
    .distinct()

processados_aux_set = {row.nome_arquivo for row in df_processados_aux.collect()}

arquivos_novos_aux = [
    f for f in arquivos_csv
    if os.path.basename(f) not in processados_aux_set
]

print(f"Arquivos auxiliares novos: {len(arquivos_novos_aux)}")

In [0]:
expected_cols_map = {}

for file_path in arquivos_novos_aux:
    try:
        nome_arquivo = os.path.basename(file_path)
        nome_delta = nome_arquivo.replace(".csv", "").lower()

        print(f"\n Processando: {nome_arquivo}")

        expected_cols = expected_cols_map.get(nome_delta)

        # Leitura robusta com aspas
        df_ok, df_corrupt, header_cols = read_csv_with_quotes(
            spark=spark,
            input_path=file_path,
            delimiter=";",
            encoding="iso-8859-1",
            recursive=False,
            path_glob_filter=None,
            header=True,
            schema=None,
            expected_cols=expected_cols,
            multiline=True,
            quote="\"",
            escape="\"",
            mode="PERMISSIVE",
            corrupt_col="_corrupt_record",
            ignore_leading_trailing_ws=True,
            quarantine_path=f"{output_path_tb_aux}/_quarentena/{nome_delta}",
            quarantine_mode="overwrite"
        )

        # Enriquecimento
        df_ok = (
            df_ok
            .withColumn("origin_path_name", input_file_name())
            .withColumn("tipo_operacao", lit(TB_AUX))
            .withColumn("ingestion_dt", current_timestamp())
        )

        # Escrita
        output_path_tb = f"{output_path_tb_aux}/{nome_delta}"

        (
            df_ok
            .write
            .format("delta")
            .mode("append")
            .save(output_path_tb)
        )

        # Métricas
        corrupt_count = df_corrupt.count() if df_corrupt is not None else 0
        ok_count = df_ok.count()

        print(f"Delta criado: {nome_delta}")
        print(f"Registros OK: {ok_count} | Corrompidos: {corrupt_count}")

        # Registrar controle
        spark.sql(f"""
            INSERT INTO bronze.arquivos_processados_balancacomercial
            VALUES (
                '{file_path}',
                '{nome_arquivo}',
                '{TB_AUX}',
                'N/A',
                'N/A',
                current_timestamp(),
                {ok_count}
            )
        """)

    except Exception as e:
        print(f"ERRO ao processar {file_path}: {e}")

print("\nIngestão de tabelas auxiliares concluída")

### Resumo da ingestão

In [0]:
print("=" * 60)
print("RESUMO DA INGESTÃO INCREMENTAL")
print("=" * 60)

df_resumo = spark.sql("""
    SELECT
        tipo_operacao,
        COUNT(*) as total_arquivos,
        SUM(num_registros) as total_registros,
        MAX(data_processamento) as ultima_ingestao
    FROM bronze.arquivos_processados_balancacomercial
    GROUP BY tipo_operacao
    ORDER BY tipo_operacao
""")

df_resumo.display()

print("\n Ingestão finalizada com sucesso!")